In [1]:
from Rings import Zomega, Ztau, Exact_U
from norm_equation import easy_factor, easy_solveable, solve_norm_equation
from exact_synthesize import exact_synthesize, exact_synthesize_weave
from Circuits import FTCircuit, Braid
from mpmath import mp, ceil, floor
import cmath
import math
import random
import numpy as np

In [2]:
'''
Set computation depth
'''
mp.dps = 100

In [3]:
'''
Define global variables
'''

phi = (mp.sqrt(mp.mpf(5)) + 1)/2
tau = (mp.sqrt(mp.mpf(5)) - 1)/2

omega = Zomega(0,1,0,0)
T = Exact_U(Zomega(1,0,0,0), Zomega(0,0,0,0), 6)
F = Exact_U(Zomega(0,0,1,-1), Zomega(1,0,0,0), 0)

sigma_1 = Exact_U(omega**6, Zomega(0,0,0,0), 4)
sigma_2 = F * sigma_1 * F

def Rz(angle):
    return np.array([
        [cmath.exp(-0.5j*angle), 0],
        [0, cmath.exp(0.5j*angle)]
    ])

def Pauli_X():
    return np.array([
        [0, 1],
        [1, 0]
    ])

def F():
    return np.array([
        [tau, math.sqrt(tau)],
        [math.sqrt(tau), -1*tau]
    ])

In [4]:
'''
Implement random sample
'''
FibArray = [1, 1]
 
def Fib(n):
    if n<0:
        print("Incorrect input")
    elif n<= len(FibArray):
        return FibArray[n-1]
    else:
        temp_fib = Fib(n-1)+Fib(n-2)
        FibArray.append(temp_fib)
        return temp_fib    

def approx_real(x,n):
    p, q = Fib(n), Fib(n+1)
    u, v = (-1)**(n+1)*Fib(n), (-1)**n * Fib(n-1)
    c = round(x*q)
    a = c*v + p * round(c*u/q)
    b = c*u - q * round(c*u/q)

    return Ztau(a, b)

def random_sample(theta, epsilon, r):
    C = mp.sqrt(phi/(4*r))
    m = int(float(mp.nstr(mp.ceil(mp.log(C*epsilon*r, tau)), 10))) + 1
    N = int(float(mp.nstr(mp.ceil(phi**m), 10)))

    y_min = r*phi**m *(mp.sin(theta) - epsilon*(mp.sqrt(4-epsilon**2)*mp.cos(theta) + epsilon*mp.sin(theta))/2)
    y_max = r*phi**m *(mp.sin(theta) + epsilon*(mp.sqrt(4-epsilon**2)*mp.cos(theta) - epsilon*mp.sin(theta))/2)
    x_max = r*phi**m *((1 - epsilon**2/2)*mp.cos(theta) - epsilon*mp.sqrt(1 - epsilon**2/4)*mp.sin(theta))
    x_c = x_max - r*epsilon**2*phi**m/(4*mp.cos(theta))

    j = random.randint(1, N-1)
    y = y_min + j*(y_max - y_min)/N
    a_y = approx_real(y/mp.sqrt(2-tau), m)
    x = x_c - (a_y.value()*mp.sqrt(2-tau) - y_min)*mp.tan(theta)
    a_x = approx_real(x,m)

    return Zomega.fromTau(a_x) + Zomega(-1,2,-1,1)*Zomega.fromTau(a_y)

In [5]:
def compile_R_rotation(angle, epsilon):
    epsilon = epsilon * epsilon
    C = mp.sqrt(phi/4)
    m = int(float(mp.nstr(mp.ceil(mp.log(C*epsilon, tau)), 10))) + 1
    theta = 0
    for k in range(10):       
        adjusted_angle = (-angle/2 - mp.pi*k/5) - floor((-angle/2 - mp.pi*k/5)/(2*mp.pi)) * (2*mp.pi)
        if adjusted_angle >= 0 and adjusted_angle <= mp.pi/5:
            theta = adjusted_angle
            break
    
    not_found, u, v = True, 0, 0

    while not_found:
        u0 = random_sample(theta, epsilon, 1)
        phi_t = Ztau(1, 1)
        xi = phi_t*(phi_t**(2*m) - Ztau.fromOmega(u0*u0.star()))
        factors = easy_factor(xi)
        if easy_solveable(factors):
            not_found = False
            u = Zomega(0,1,0,0)**k * Zomega.fromTau(Ztau(0,1))**m * u0
            v = Zomega.fromTau(Ztau(0,1))**m * solve_norm_equation(xi)
    
    print("u, v = " + str(u) + ", " + str(v))
    print("distance = " + str(mp.sqrt(1 - 0.5 * abs((u.value()*mp.exp(mp.mpc(1j)*angle/2) + (u.star().value()*mp.exp(mp.mpc(-1j)*angle/2)))))))
    #print(mp.log(abs(u.a)))
    Circuit = exact_synthesize(Exact_U(u, v, 5))

    return Circuit

In [7]:
angle = mp.pi
print(Rz(angle))
FT = compile_R_rotation(angle, 10**(-6))
braid = str(FT.toBraid())
print(Braid.shorten_braid_string(braid))

[[6.123234e-17-1.j 0.000000e+00+0.j]
 [0.000000e+00+0.j 6.123234e-17+1.j]]
u, v = (1956437049930420306546432858390528 + -6069708505166260617571786970628096ω + 6655413019526331252527854543437824ω² + -2904126861529265461874877496557568ω³), (3187587060607207580513211598388109 + -4919510717235477284187678423958279ω + 2802311342344542489765165091763510ω² + 238113488926355613806891487557388ω³)
distance = 0.0000007666339959453191389398325786425305600009272495354123553632854561014456965499626393599005404950094577
(314, 'exp(0.0πi)3333444333444111441112222211111222211122333444433344443322221112223332221144411112221111444111114441111144433222233222233222233334443334441112221144433332221112233222233332221111222211111222221111122211144441111223334444333444333222233332223334444111222111122211444111444333322211144111223333222233323333')
